In [1]:
!which python

/Users/alelevier/Documents/github/ssd-pytorch/venv/bin/python


In [2]:
import enum
import math
import pdb
import platform
import time

import cv2
import numpy as np
import torch
import torch.nn.functional as F
from matplotlib import patches, patheffects
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from ssdmultibox import utils
from ssdmultibox.bboxer import Bboxer, TensorBboxer
from ssdmultibox.config import cfg
from ssdmultibox.criterion import SSDLoss
from ssdmultibox.datasets import PascalDataset, TrainPascalFlatDataset, device
from ssdmultibox.models import SSDModel
from ssdmultibox.plotting import *
from ssdmultibox.predict import Predict
from ssdmultibox.utils import open_image

import matplotlib.pyplot as plt
plt.ion()

In [3]:
device

device(type='cpu')

In [4]:
losses = {'total':[], 'loc':[], 'conf':[]}

In [5]:
model = SSDModel().to(device)

In [6]:
LR = 1e-2

criterion = SSDLoss(alpha=.2)
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=0.0005)
current_time = time.time()

In [7]:
TrainPascalFlatDataset.__len__ = lambda self: 4

train_dataset = TrainPascalFlatDataset()

len(train_dataset)

4

In [8]:
train_dataset

In [9]:
NUM_WORKERS = 0
BATCH = 4
SHUFFLE = False
    
dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=SHUFFLE, num_workers=NUM_WORKERS)

vars(dataloader)

{'dataset': <ssdmultibox.datasets.TrainPascalFlatDataset at 0x1423ef748>,
 'batch_size': 4,
 'num_workers': 0,
 'collate_fn': <function torch.utils.data.dataloader.default_collate(batch)>,
 'pin_memory': False,
 'drop_last': False,
 'timeout': 0,
 'worker_init_fn': None,
 'sampler': <torch.utils.data.sampler.SequentialSampler at 0x142416048>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x142416080>,
 '_DataLoader__initialized': True}

In [10]:
EPOCHS = 1

for epoch in range(EPOCHS):
    steps = math.ceil(len(train_dataset) / BATCH)
    for step in tqdm_notebook(range(steps)):
        image_ids, ims, gt_bbs, gt_cats = next(iter(dataloader))

        # put data on device
        ims, gt_bbs, gt_cats = PascalDataset.to_device(ims, gt_bbs, gt_cats)

        # zero out gradients
        optimizer.zero_grad()

        # forward pass
        preds = model(ims)
        loss, loc_loss, conf_loss = criterion(preds, (gt_bbs, gt_cats))

        # backward pass
        loss.backward()
        optimizer.step()

        # stats
        if step % 5 == 0:
            print('epoch: {} step: {} loss: {:.4f} time: {:.4f}'.format(
                epoch, step, loss.item(), time.time() - current_time))
            current_time = time.time()
            losses['total'].append(loss.item())
            losses['loc'].append(loc_loss.item())
            losses['conf'].append(conf_loss.item())

pos_loss: 112.7236 neg_hard_mining_loss: 354.0742
n: 37.0 bbs_loss: 542.5619 cats_loss: 466.7977
epoch: 0 step: 0 loss: 27.2800 time: 6.7151



In [11]:
bbs_preds, cats_preds = preds

In [12]:
idx = 0
cls_id = 6
dataset = train_dataset

In [13]:
# targets
get_targets(gt_cats, idx)

(tensor([[100.0000, 100.0000, 200.0000, 200.0000],
         [ 50.0000, 100.0000, 250.0000, 200.0000],
         [100.0000, 100.0000, 300.0000, 200.0000],
         [ 97.2954, 100.0000, 202.7046, 200.0000]]), tensor([6, 6, 6, 6]))

In [14]:
get_targets(gt_cats, idx, bbs_preds)

(tensor([[100.3507,  99.5252, 199.9233, 200.0656],
         [ 49.4617, 100.4373, 249.2867, 200.1265],
         [100.2074,  99.7542, 299.7056, 200.5531],
         [ 97.2666, 100.0502, 203.4629, 199.5765]], grad_fn=<TakeBackward>),
 tensor([6, 6, 6, 6]))

In [15]:
# plot_anchor_bbs(dataset, image_ids, idx, gt_cats)

In [16]:
# plot_anchor_bbs(dataset, image_ids, idx, gt_cats)

In [17]:
# plot_nms_preds(dataset, image_ids, idx, preds, limit=5)

In [18]:
# plot_nms_single_preds(dataset, image_ids, idx, cls_id, preds, limit=5)